In [7]:
import pandas as pd 
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

# Carregar tabela de CPFs
tabela = pd.read_excel("CPFS (1).xlsx")

# Configurar o navegador (uma instância única)
navegador = webdriver.Chrome()

# Iterar sobre os CPFs na tabela
for i, cpf in enumerate(tabela["CPF"]):
    descricao = tabela.loc[i, "DESCRICAO"]

    # Acessar o site
    navegador.get("https://www.igrejacristamaranata.org.br/ebd/participacoes/")

    # Esperar até o campo CPF estar presente e preenchê-lo
    elemento_usuario = WebDriverWait(navegador, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="icmEbdNacionalForm"]/div[2]/div/div[1]/input'))
    )
    elemento_usuario.send_keys(cpf)

    # Rolar para o botão "Participação" e clicar
    botao_participacao = WebDriverWait(navegador, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="icmEbdNacionalForm"]/div[5]/div/div[1]/input[2]'))
    )
    ActionChains(navegador).move_to_element(botao_participacao).perform()
    botao_participacao.click()

    # Esperar até o campo de mensagem e preencher com a descrição
    elemento_mensagem = WebDriverWait(navegador, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="icmEbdNacionalForm"]/div[5]/div/div[2]/div/div[2]/div[1]/p'))
    )
    elemento_mensagem.send_keys(descricao)

   # Localizar a caixa "Aceito os Termos"
    termo = WebDriverWait(navegador, 20).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="aceitoOsTermos"]'))
    )

    # Rolar para o elemento
    navegador.execute_script("arguments[0].scrollIntoView(true);", termo)
    time.sleep(1)  # Pequena pausa

    # Tentar clicar usando Selenium ou JavaScript, caso necessário
    try:
        termo.click()
    except ElementClickInterceptedException:
        navegador.execute_script("arguments[0].click();", termo)

    # Submeter o formulário
    botao_enviar = WebDriverWait(navegador, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="icmEbdNacionalForm"]/button'))
    )
    botao_enviar.click()

    # Aguardar para evitar bloqueio de requisições
    time.sleep(3)

# Fechar o navegador após o loop
navegador.quit()
